<a href="https://colab.research.google.com/github/jesusrevilla/mineria-de-datos/blob/main/primer-parcial/05_Correlacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minería de Datos: **Correlación** con pandas (Google Colab)

**Autor:** (tu nombre)  **Objetivo:** Entender y aplicar correlaciones (Pearson, Spearman, Kendall) en un dataset real utilizando `pandas`.

> 💡 Ejecuta las celdas de arriba hacia abajo (Shift+Enter).

## 🎯 Objetivos de aprendizaje
1. Comprender qué miden **Pearson**, **Spearman** y **Kendall** y cuándo usar cada uno.
2. Calcular y visualizar **matrices de correlación** con `pandas` y `seaborn`.
3. Detectar **multicolinealidad** con **VIF** y tomar decisiones.
4. Tratar **faltantes** y **atípicos** que sesgan la correlación.
5. Extraer **insights** para modelado y toma de decisiones.

## 🧰 Prerrequisitos (librerías)
En Colab ya vienen instaladas: `pandas`, `numpy`, `seaborn`, `matplotlib`, `scipy`, `statsmodels`.
Si trabajas local, instala con:
```bash
pip install pandas numpy seaborn matplotlib scipy statsmodels
```

In [ ]:
# # Minería de Datos: Correlación con pandas (Colab)
# Autor: (tu nombre)
# Objetivo: entender y aplicar correlaciones (Pearson, Spearman, Kendall) en un dataset real.

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import pearsonr, spearmanr, kendalltau
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

sns.set(style="whitegrid")
pd.set_option('display.float_format', lambda x: f'{x:.3f}')


In [ ]:
# Cargar dataset de pingüinos desde seaborn
df = sns.load_dataset("penguins")

print("Dimensiones:", df.shape)
display(df.head())
display(df.describe(include='all'))


In [ ]:
# Limpieza básica: faltantes, duplicados, tipos
faltantes = df.isna().sum().sort_values(ascending=False)
print("Faltantes por columna:
", faltantes)

# Eliminar duplicados si los hubiera
antes = df.shape[0]
df = df.drop_duplicates().copy()
despues = df.shape[0]
print(f"Duplicados eliminados: {antes - despues}")

# Imputación simple
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode().iloc[0])

print("Faltantes restantes:", int(df.isna().sum().sum()))


In [ ]:
# Exploración visual rápida
df[num_cols].hist(figsize=(12, 8), bins=20)
plt.suptitle("Distribuciones de variables numéricas", y=1.02)
plt.show()

sns.pairplot(df.sample(min(300, len(df))), vars=num_cols, hue="species", diag_kind="hist")
plt.suptitle("Relaciones bivariadas por especie", y=1.02)
plt.show()


In [ ]:
# Matrices de correlación (Pearson, Spearman, Kendall)
corr_pearson = df[num_cols].corr(method="pearson")
corr_spearman = df[num_cols].corr(method="spearman")
corr_kendall = df[num_cols].corr(method="kendall")

def plot_corr_heatmap(corr, title):
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1, square=True, fmt=".2f")
    plt.title(title)
    plt.show()

plot_corr_heatmap(corr_pearson, "Matriz de correlación (Pearson)")
plot_corr_heatmap(corr_spearman, "Matriz de correlación (Spearman)")
plot_corr_heatmap(corr_kendall, "Matriz de correlación (Kendall)")


In [ ]:
# Correlaciones puntuales con significancia
x = df["bill_length_mm"]
y = df["body_mass_g"]

r_p, p_p = pearsonr(x, y)
r_s, p_s = spearmanr(x, y)
r_k, p_k = kendalltau(x, y)

print(f"Pearson r={r_p:.3f}, p-valor={p_p:.4g}")
print(f"Spearman ρ={r_s:.3f}, p-valor={p_s:.4g}")
print(f"Kendall τ={r_k:.3f}, p-valor={p_k:.4g}")


In [ ]:
# Efecto de atípicos (outliers) en Pearson vs Spearman
df_out = df.copy()
df_out.loc[df_out.sample(1, random_state=42).index, "body_mass_g"] *= 3

r_p_o, _ = pearsonr(df_out["bill_length_mm"], df_out["body_mass_g"])
r_s_o, _ = spearmanr(df_out["bill_length_mm"], df_out["body_mass_g"])

print(f"Sin outlier -> Pearson: {r_p:.3f}, Spearman: {r_s:.3f}")
print(f"Con outlier -> Pearson: {r_p_o:.3f}, Spearman: {r_s_o:.3f}")

fig, axs = plt.subplots(1,2, figsize=(12,5), sharex=True, sharey=True)
sns.scatterplot(x=df["bill_length_mm"], y=df["body_mass_g"], ax=axs[0])
axs[0].set_title("Sin outlier")
sns.scatterplot(x=df_out["bill_length_mm"], y=df_out["body_mass_g"], ax=axs[1], color="tomato")
axs[1].set_title("Con outlier artificial")
for ax in axs:
    ax.set_xlabel("bill_length_mm")
    ax.set_ylabel("body_mass_g")
plt.tight_layout()
plt.show()


In [ ]:
# Correlación con variables categóricas mediante one-hot encoding
df_enc = pd.get_dummies(df, columns=[c for c in df.columns if c in (set(df.columns) - set(df.select_dtypes(include=[np.number]).columns))], drop_first=True)
corr_all = df_enc.corr(method="pearson")
# Seleccionamos columnas originales numéricas para filas
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
def plot_corr_heatmap(corr, title):
    plt.figure(figsize=(10, max(6, len(num_cols)*0.6)))
    sns.heatmap(corr, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
    plt.title(title)
    plt.show()

subset = corr_all.loc[num_cols]  # filas: numéricas; columnas: todas (incluye dummies)
plot_corr_heatmap(subset, "Correlación de numéricas vs. variables (incluyendo dummies)")


In [ ]:
# Multicolinealidad: VIF
X = df_enc.drop(columns=["body_mass_g"], errors='ignore')
X = sm.add_constant(X)

vif_df = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
}).sort_values("VIF", ascending=False)

display(vif_df.head(15))
print("Regla práctica: VIF > 5 (o 10) sugiere multicolinealidad importante")


In [ ]:
# Correlación parcial entre X e Y controlando variables Z
def partial_corr(x, y, controls, data):
    Xc = sm.add_constant(data[controls])
    model_x = sm.OLS(data[x], Xc).fit()
    model_y = sm.OLS(data[y], Xc).fit()
    rx = model_x.resid
    ry = model_y.resid
    r, p = pearsonr(rx, ry)
    return r, p

r_partial, p_partial = partial_corr("bill_length_mm", "body_mass_g",
                                    controls=["flipper_length_mm", "bill_depth_mm"],
                                    data=df)
print(f"Correlación parcial (controles: flipper_length_mm, bill_depth_mm): r={r_partial:.3f}, p={p_partial:.4g}")


In [ ]:
# Reporte: top pares por |correlación| en Pearson
corr_pearson = df.select_dtypes(include=[np.number]).corr(method="pearson")
corr_abs = corr_pearson.abs()
upper = corr_abs.where(np.triu(np.ones(corr_abs.shape), k=1).astype(bool))
top_pairs = (
    upper.stack()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={"level_0": "var1", "level_1": "var2", 0: "abs_corr"})
)
display(top_pairs.head(10))

print("
Guía de interpretación:
- |r| ~ 0.1: débil; ~0.3: moderada; >=0.5: fuerte (reglas generales).
- Pearson: lineal; sensible a outliers y escala.
- Spearman/Kendall: monótonas; más robustas; no requieren normalidad.
- Correlación ≠ causalidad. Considera confusores y el dominio del problema.
- En modelado, revisa multicolinealidad (VIF) para eliminar/combinar variables.
")


## 🧠 Teoría breve
- **Correlación**: fuerza y dirección de la relación entre dos variables.
  - **Pearson (r)**: relación **lineal** entre variables numéricas; sensible a outliers.
  - **Spearman (ρ)**: correlación de **rangos**; capta relaciones **monótonas**.
  - **Kendall (τ)**: basado en pares concordantes/discordantes; útil con datos ordinales.
- **Buenas prácticas**: Visualiza (scatter/pairplot), trata faltantes y atípicos, y recuerda que **correlación no implica causalidad**.


## 🎯 Actividades sugeridas
1. Cambia la variable objetivo a `flipper_length_mm` y repite VIF.
2. Estratifica por `species` y compara matrices de correlación.
3. Introduce outliers en `bill_depth_mm` y compara Pearson vs Spearman.
4. Calcula correlación parcial entre `flipper_length_mm` y `body_mass_g` controlando por `species` (usa dummies).
5. Genera un reporte con los 5 pares de mayor correlación positiva y negativa e interpreta.
